In [1]:
import torch
import torch.utils.data as data
import os
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.pyplot as plt
%matplotlib inline

class MyDatasets(data.Dataset):
    def __init__(self, root, data_folder, mode='train', transform=None, target_transform=None):
        self.root = root
        self.data_folder = data_folder
        self.mode = mode
        self.transform = transform
        self.target_transform = target_transform
        
        if self.mode == 'train':
            self.train_data, self.train_label = self.load_mnist(os.path.join(self.root, data_folder))
        else:
            self.test_data, self.test_label = self.load_mnist(os.path.join(self.root, data_folder))

    def load_mnist(self, mnist_path):
        df = pd.read_csv(os.path.join(mnist_path, 'train.csv'))
        if self.mode == 'train':
            df  = df[:40000]
        else:
            df = df[40000:]
        df = np.array(df, np.uint8)
        img_datas = df[:, 1:]
        label_datas = df[:, 0]
        
        return img_datas, label_datas
    
    def __getitem__(self, index):
        if self.mode == 'train':
            img, label = self.train_data[index], self.train_label[index]
        else:
            img, label = self.test_data[index], self.test_label[index]
        
        return img, label
    
    def __len__(self):
        if self.mode == 'train':
            return len(self.train_data)
        else:
            return len(self.test_data)

class Mnist_network(nn.Module):
    
    def __init__(self):
        super(Mnist_network, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32, eps=0.001)
        self.pool1 = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(32, 64, 3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64, eps=0.001)
        self.pool2 = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(64*7*7, 200)
        self.bn3 = nn.BatchNorm1d(200, eps=0.001)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(200, 10)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.pool2(x)
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        
        return x
    
def test(test_loader, model):
    
    model.eval()
    correct = 0
    for index, (images, labels) in enumerate(test_loader):
        images = np.reshape(images.float(), (images.shape[0], 1, 28, 28))
        images = Variable(images)
        labels = Variable(labels).long()
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()
        outputs = model(images)
        predict = torch.max(outputs, 1)[1].long()
        if predict.data.cpu().numpy() == labels.data.cpu().numpy():
            correct += 1
    accuracy = correct * 1.0 / len(test_loader)
    
    return accuracy

def init_weight(model, dev=0.01):
    if isinstance(model, list):
        for m in model:
            init_weight(m, dev)
    else:
        for m in model.modules():
            if isinstance(m, nn.Conv2d):
                m.weight.data.normal_(0.0, dev)
            else if isinstance(m, nn.Linear):
                m.weight.data.normal_(0.0, dev)
        

if __name__=='__main__':
    
    train_mode = 'train'
    test_mode = 'test'
    
    train_dataset = MyDatasets('./', 'data', train_mode)
    test_dataset = MyDatasets('./', 'data', test_mode)
    
    train_loader = data.DataLoader(dataset=train_dataset, batch_size=128, shuffle=True)
    test_loader = data.DataLoader(dataset=test_dataset, batch_size=1, shuffle=True)
    
    model = Mnist_network()
    
    print(model)
    
    if torch.cuda.is_available():
        model = model.cuda()
    
    init_weight(model)
    
    learning_rate = 0.001
    epoch_num = 200
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    
    best_epoch = 1
    max_accuracy = -999999.0
    accuracy_history = []
    
    for epoch in range(epoch_num):
        model.train()
        for index, (images, labels) in enumerate(train_loader):
            images = np.reshape(images.float(), (images.shape[0], 1, 28, 28))
            images = Variable(images)
            labels = Variable(labels).long()
            
            if torch.cuda.is_available():
                images = images.cuda()
                labels = labels.cuda()
            
            optimizer.zero_grad()
            digital_pred = model(images)
            loss = F.cross_entropy(digital_pred, labels)
            loss.backward()
            optimizer.step()
            
        accuracy = test(test_loader, model)
        accuracy_history.append(accuracy)
        if accuracy > max_accuracy:
            max_accuracy = accuracy
            best_epoch = epoch
        print('Current epoch: {:d} / {:d}. Accuracy is {:.2f}%'.format(epoch+1, epoch_num, accuracy * 100.0))
    
    plt.plot(accuracy_history)
    plt.show()
    print('train end')
    print('Best train epoch is {:d}, max accuracy is {:.2f}%'.format(best_epoch, max_accuracy * 100.0))
    
        
        

<bound method Mnist_network.modules of Mnist_network (
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True)
  (pool1): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True)
  (pool2): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (fc1): Linear (3136 -> 200)
  (bn3): BatchNorm2d(200, eps=0.001, momentum=0.1, affine=True)
  (relu): ReLU ()
  (fc2): Linear (200 -> 10)
)>


NameError: name 'fang' is not defined